In [87]:
# data processing
import skvideo.io
import numpy as np
from scipy.ndimage import gaussian_filter, uniform_filter1d
from pathlib import Path

# plotting
import matplotlib.pylab as plt

# normalize values of array to span 0 to 255
norm_vid = lambda v: ((v - v.min()) / (v.max() - v.min()) * 255).astype(int)
# crop video to d x d square around point x,y
crop_vid = lambda v, x, y, d: v[:, y-d//2:y+d//2, x-d//2:x+d//2]
# calculate gradient between frames
grad_vid = lambda v: np.gradient(v, axis=0)
# colapse 2D image to 1D vector
flat_vid = lambda v: np.reshape(v, (v.shape[0], -1))

# load and write video files
write_vid = lambda path, arr: skvideo.io.vwrite(str(path), norm_vid(arr), inputdict={'-framerate': '30'})
read_vid = lambda path: skvideo.io.vread(str(path))[:,:,:,0]

def adjust_crop(vid, x, y, d):
    '''Check if crop area is within video and adjust accordingly


    Parameters
    ----------
    vid : numpy.array
        video
    x : int
        x coordinate of the center in the cropped video
    y : int
        x coordinate of the center in the cropped video
    d : int
        dimension in pixel of the cropped video

    Returns
    -------
    x : int
        adjusted x coordinate of the center in the cropped video
    y : int
        adjusted y coordinate of the center in the cropped video
    '''

    _, res_y, res_x = vid.shape # get video resolution
    
    x, y = np.max([d // 2, x]), np.max([d // 2, y]) # adjust lower bound
    x, y = np.min([res_x - (d // 2), x]), np.min([res_y - (d // 2), y]) # adjust upper bound
    
    return x, y
    

def generate_gradient(p, x, y, d):
    '''Read video file from disc, crop and calculate frame-to-frame gradient

    Parameters
    ----------
    p : str
        Path to video file
    x : int
        x coordinate of the center in the cropped video
    y : int
        x coordinate of the center in the cropped video
    d : int
        dimension in pixel of the cropped video

    Returns
    -------
    grad : numpy.array
        cropped video with the frame-to-frame gradient
    '''
    p = Path(p)

    print('INFO: Reading video file {}'.format(p))
    raw = read_vid(str(p))

    print('INFO: Checking crop area: {} pxl square around x={} and y={}'.format(d, y, y))
    x, y = adjust_crop(raw, x, y, d)
    print('INFO: Cropping area to:   {} pxl square around x={} and y={}'.format(d, x, y))
    vid = crop_vid(raw, x, y, d)
    print('INFO: Calculating frame-to-frame gradient')
    grad = grad_vid(vid)

    p_crop = p.with_name(p.stem + '_crop.mp4')
    print('SAVING: {}'.format(p_crop))
    write_vid(p_crop, vid)

    p_grad = p.with_name(p.stem + '_grad.mp4')
    print('SAVING: {}'.format(p_grad))
    write_vid(p_grad, grad)

    return grad

def load_gradient(p):
    '''Load gradient video file from disk

    Parameters
    ----------
    p : str
        Path to parent video i.e. somevideo.avi

    Returns
    -------
    grad : numpy.array
        cropped video with the frame-to-frame gradient
    '''

    p = Path(p)

    p_crop = p.with_name(p.stem + '_crop.mp4')
    print('    INFO: Reading video file {}'.format(p_crop))
    vid = read_vid(p_crop)

    print('    INFO: Calculating frame-to-frame gradient')
    grad = grad_vid(vid)

    return grad


def plot_hist(ax, arr, nbins=100, thresh=10):
    '''Plot binary histogram for each frame in video

    Parameters
    ----------
    ax : matplotlib axes
        axes to plot to
    arr : numpy.array
        video
    nbins : int, optional
        Number of bins, by default 100
    thresh : float
        plot line at +/- thresh to visualize threshold
    '''
    arr = flat_vid(arr)

    bins = np.linspace(arr.min(), arr.max(), nbins) # define bins
    fun = lambda x: np.histogram(x, bins=bins)[0] # function to apply to each frame
    hist = np.apply_along_axis(fun, 1, arr) # apply function to vid
    hist = np.clip(hist, 0, 1) # show every non-zero bin as 'clip'

    # plot colormes colormesh
    xedges, yedges = range(hist.shape[0]), bins[:-1]
    X, Y = np.meshgrid(xedges, yedges)
    ax.pcolormesh(X, Y, hist.T)
    # plot thresholds
    ax.axhline(thresh, color='white')
    ax.axhline(-thresh, color='white')

    ax.set_ylabel('binary pixel gradients')
    ax.set_title('binned pixel gradient (binary)')
    ax.set_xlabel('frame')

def plot_sum(ax, arr, thresh=10, sigma=0, avg=1, ylim=(None, None)):
    '''Plot sum of bins in pixel gradient above threshold for each frame in video

    Parameters
    ----------
    ax : matplotlib axes
        axes to plot to
    arr : numpy.array
        video
    thresh : float
        do not count pixels with values above/below +/- thresh
    sigma : float, default 0
        standard deviation of Gaussian kernel. sigma=0 implies no smoothing
    avg : int, default 1
        width of moving average (boxcar kernel). avg=1 implies no averaging
    ylim : tup, default (None, None)
        tuple with ymin and ymax, only affects the plot
    '''
    arr = flat_vid(arr)

    y = (np.abs(arr) > thresh).sum(axis=1)
    y = uniform_filter1d(y, size=avg)
    y = gaussian_filter(y, sigma=sigma)

    # plot
    ax.plot(y)
    ax.set_ylim(ylim)
    ax.set_xlabel('frame')
    ax.set_ylabel('sum of pixels')
    ax.set_title('pixels above and below threshold {}'.format(thresh))

    ymin, ymax = ax.get_ylim()
    z = y.copy()
    z[z != 0] = ymax - 0.02 * (ymax - ymin)
    z[z == 0] = ymin + 0.02 * (ymax - ymin)
    x = np.arange(len(z))
    ax.scatter(x, z, color='C1', marker=',', s=1)
    
    return y

def get_rep(x):
    '''Calculate zero and non-zero repeats in array

    Parameters
    ----------
    x : np.array
        array with zero and non-zero elements

    Returns
    -------
    txt : list
        list with strings describing zero and non-zero repeats
    '''
    x[x != 0] = 1
    split = np.split(x, np.flatnonzero(np.diff(x) != 0) + 1)

    fi = 0
    txt = []
    for s in split:
        state = 'on' if s.any() else 'off'
        ff = fi + len(s) - 1
        out = 'Frames {:5d} to {:5d}: {:>3}'.format(fi + 1, ff + 1, state)
        # print('    {}'.format(out))
        txt.append(out)
        fi = ff + 1

    return txt

def analyze_gradient(p, grad, thresh, sigma, avg):
    '''Extract still and moving frames from frame-to-frame gradient video 

    Parameters
    ----------
    p : str
        Path to partent video i.e. somevideo.avi
    grad : numpy.array
        frame-to-frame gradien video
    thresh : float
        ignore pixels above/below this positive/negative value
    sigma : float
        standard deviation of the Gaussian kernel to smooth across frames
    '''
    
    p = Path(p)
    
    # heatmap of binary gradients
    print('PLOTTING: clipped pixel gradients')
    fig, ax = plt.subplots()
    plot_hist(ax, grad, thresh=thresh)

    p_grad = p.with_name(p.stem + '_hist.png')
    print('  SAVING: {}'.format(p_grad))
    fig.savefig(p_grad)

    # sum pixels with large gradient
    print('PLOTTING: sum of pixel with large gradients')
    fig, ax = plt.subplots()
    pxl_sum = plot_sum(ax, grad, thresh=thresh, sigma=sigma, avg=avg)

    p_sum = p.with_name(p.stem + '_sum.png')
    print('  SAVING: to {}'.format(p_sum))
    fig.savefig(p_sum)

    on, off = (pxl_sum != 0).sum(), (pxl_sum == 0).sum()
    print('    INFO: There are {} on and {} off frames'.format(on, off))
    arr = np.vstack((np.arange(len(pxl_sum)), np.clip(pxl_sum, 0, 1))).T
    p_still = p.with_name(p.stem + '_stillframes.csv')
    print('  SAVING: {}'.format(p_still))
    np.savetxt(p_still, arr, delimiter=',', fmt='%d', header='frame,on/off', comments='')

    txt = get_rep(pxl_sum)
    print('    INFO: There are {} repeats'.format(len(txt)))
    p_rep = p.with_name(p.stem + '_rep.txt')
    print('  SAVING: {}'.format(p_rep))
    with open(p_rep, 'w') as f:
        f.write('There are {} on and {} off frames\n'.format(on, off))
        f.write('\n'.join(txt))

# How to use 

## step 1: crop video and calculate gradient
The function `generate_gradient` requires:
- path to the `somevideo.avi` file
- x and y coordinate of the to-be-cropped field of view (i.e. the position of the fly)
- the size of the cropped square in pixel

It generates the files
- the cropped video `somevideo_crop.avi`
- the cropped pixel gradient video `somevideo_grad.avi`

Afterward, inspect the cropped video, if it correctly captures the fly.

In [7]:
p = r'C:\somevideo.avi'
grad = generate_gradient(p, x=495, y=480, d=400)

To avoid loading the original video, call `load_gradient`, this will load `somevideo_crop.avi` and generate the gradient from it.
The gradient need to be regenerated, since the pixels in `somevide_grad.avi` are normalized.

In [ ]:
p = r'C:\somevideo.avi'
grad = load_gradient(p)

## step 2: analyze the gradient video
The function `analyze_gradient` uses the pixel gradient video generated in step 1 (`grad`). It only requires the path to `somevideo.avi`, because it will generate the output file names as `somevideo_somethingelse.avi`.

It calculates and plots for each frame
- a binary histogram, which means that either a value does not occur (0) or it occurs one or more times (1), `somevideo_hist.png`
- the sum of non-zero elements in the histogram above a given threshold, `somevideo_sum.png`
Based on the sum of pixels, frames are classified as still or moving, which is visualized in `somevideo_sum.png` as well as printed in `somevideo_rep.txt`.

Adjust the parameters `thresh` and `sigma`/`avg` to get a clean separation.
The threshold is visualized in the `somevideo_hist.png` and is controlled via the `thresh` arguement. 
The argument `sigma` controls the width of a Gaussian kernel to smooth across frames in `somevideo_sum.png` (set `sigma=0` to disable smoothing).
The argument `avg` controls the width of a boxcar kernel to average across frames (set `avg=1` to disable averaging).


In [ ]:
analyze_gradient(p, grad, thresh=15, sigma=0, avg=1)

# batch-process multiple videos
Important: each video requires the position of the fly

In [ ]:
args_list = [
    (r'C:\somevideo1.avi', 100, 210, 400),
    (r'C:\somevideo2.avi', 200, 220, 400),
    (r'C:\somevideo3.avi', 300, 230, 400),
]
for args in args_list:
    p, x, y, d = args
    grad = generate_gradient(p, x, y, d)
    analyze_gradient(p, grad, thresh=12, sigma=0, avg=1) 


# Only analysis

To do only the analysis of the gradient without loading and cropping the raw video, call


In [ ]:
p = r'C:\somevideo.avi'
grad = load_gradient(p) # step 1
analyze_gradient(p, grad, thresh=15, sigma=0, avg=1) # step 2

or in batch mode:

In [ ]:
ps = [
    r'C:\somevideo1.avi',
    r'C:\somevideo2.avi',
    r'C:\somevideo3.avi',
]

for p in ps:
    grad = load_gradient(p)
    analyze_gradient(p, grad, thresh=12, sigma=1)

# alternative, to copy-paste list of tuples:
args_list = [
    (r'C:\somevideo1.avi', 100, 210, 400),
    (r'C:\somevideo2.avi', 200, 220, 400),
    (r'C:\somevideo3.avi', 300, 230, 400),
]
for args in args_list:
    p, _, _, _ = args
    grad = load_gradient(p)
    analyze_gradient(p, grad, thresh=12, sigma=0, avg=1) 

Note that this will read the `somevideo_grad.mp4` file from disk, which is a compressed file, and therefore the results will not be exactly the same as above.